# Imports

In [42]:
import pandas as pd
import numpy as np
import xgboost as xgb
from google.colab import files
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import make_scorer, log_loss

# Carga de los data sets de test y de entrenamiento

In [43]:
test = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Test_TP2_Datos_2020-2C.csv')
train = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/Train_TP2_Datos_2020-2C.csv')

train_set_features = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/train_set_Features.csv')
test_set_features = pd.read_csv('https://raw.githubusercontent.com/FerFabbiano/tp2-organizacionDeDatos/main/test_set_Features.csv')

In [44]:
train_set_features = train_set_features.drop_duplicates(subset="Opportunity_ID")
test_set_features = test_set_features.drop_duplicates(subset="Opportunity_ID")

Separamos la variable a predecir. Esta vez vamos a utilizar el data frame procesado, que contiene el feature "Total Taxable Amount USD".

In [45]:
#X = DATA , Y = TARGET 
X1, y1 = train_set_features[['Total_Taxable_Amount_USD',
                            'Bureaucratic_Code_0_Approved', 
                            'Products_Per_Op',
                            'TRF_Per_Op',
                            'Daily_Dif_Account_Created_Opportunity_Created_Date',
                            'Daily_Dif_Delivery_Start_Delivery_End_Date',
                            'Opp_Owner_Did_Last_Modifiy',
                            'Pricing, Delivery_Terms_Approved',
                            'Bureaucratic_Code_0_Approval',
                            'Last_Modified_date>Planned_Delivery_End_Date',
                            'Op_Has_A_Valid_Quote_Expiry_Date']], train_set_features['Stage']

Dividimos el set de entrenamiento en train y test, utilizando una proporción del 30% del total de los datos para el segundo.

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

# Random Search con XGBoost

In [47]:
xg_clas = XGBClassifier(eval_metric = 'mlogloss')

In [48]:
posibles_hiperparametros = {'learning_rate':np.arange(0.01,0.3,0.1),
                              'n_estimators':np.arange(5,100,5),
                              'scale_pos_weight':np.arange(2,6,1),
                              'max_depth':np.arange(2,7,1),
                            'min_child_weight':np.arange(1,10,1),
                              'gamma':np.arange(0,0.5,0.1),
                              'subsample':np.arange(0.2,1,0.1),
                            'colsample_bytree':np.arange(0.4,0.8,0.05),
                              'colsample_bylevel':np.arange(0.4,0.8,0.05)}

LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

random_hiperparametros = RandomizedSearchCV(xg_clas, 
                                            posibles_hiperparametros, 
                                            n_iter = 10,
                                            verbose = 3,
                                            n_jobs = -1,
                                            scoring=LogLoss)
mejores_hp = random_hiperparametros.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   15.0s finished


In [49]:
mejores_hp.best_params_

{'colsample_bylevel': 0.75,
 'colsample_bytree': 0.4,
 'gamma': 0.1,
 'learning_rate': 0.21000000000000002,
 'max_depth': 5,
 'min_child_weight': 9,
 'n_estimators': 75,
 'scale_pos_weight': 2,
 'subsample': 0.6000000000000001}

# Bagging Classifier con XGBoost

In [50]:
clf = BaggingClassifier(base_estimator=XGBClassifier(**mejores_hp.best_params_),
                        n_estimators = 30, 
                        random_state = 0,
                        n_jobs = -1,
                        oob_score = True)

clf.fit(X_train, y_train)

BaggingClassifier(base_estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                               colsample_bylevel=0.75,
                                               colsample_bynode=1,
                                               colsample_bytree=0.4, gamma=0.1,
                                               learning_rate=0.21000000000000002,
                                               max_delta_step=0, max_depth=5,
                                               min_child_weight=9, missing=None,
                                               n_estimators=75, n_jobs=1,
                                               nthread=None,
                                               objective='binary:logistic',
                                               random_state=0, reg_alpha=0,
                                               reg_lambda=1, scale_pos_weight=2,
                                               seed=None, silent=None,
                     

In [51]:
predicciones_train = clf.predict_proba(X_test)

In [52]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_train)

0.45266366678783254

In [53]:
predicciones_finales = clf.predict_proba(test_set_features[['Total_Taxable_Amount_USD',
                            'Bureaucratic_Code_0_Approved', 
                            'Products_Per_Op',
                            'TRF_Per_Op',
                            'Daily_Dif_Account_Created_Opportunity_Created_Date',
                            'Daily_Dif_Delivery_Start_Delivery_End_Date',
                            'Opp_Owner_Did_Last_Modifiy',
                            'Pricing, Delivery_Terms_Approved',
                            'Bureaucratic_Code_0_Approval',
                            'Last_Modified_date>Planned_Delivery_End_Date',
                            'Op_Has_A_Valid_Quote_Expiry_Date']])

In [54]:
predicciones_exito = []
for prob in predicciones_finales:
  predicciones_exito.append(prob[1])

In [55]:
csv_kaggle = pd.DataFrame()
csv_kaggle['Opportunity_ID'] = test_set_features["Opportunity_ID"]
csv_kaggle['target'] = predicciones_exito

In [56]:
csv = csv_kaggle.to_csv('predicciones_bagging_xgboost.csv', index=False)
#files.download('predicciones_bagging_xgboost.csv')

### Resultados

Algoritmo: **XGBoost + Bagging**

Búsqueda de hiper parámetros: Aleatorio

Features analizados: 

+ Total_Taxable_Amount_USD
+ Bureaucratic_Code_0_Approved
+ Products_Per_Op
+ TRF_Per_Op
+ Daily_Dif_Account_Created_Opportunity_Created_Date               
+ Daily_Dif_Delivery_Start_Delivery_End_Date
+ Opp_Owner_Did_Last_Modifiy
+ Pricing, Delivery_Terms_Approved
+ Bureaucratic_Code_0_Approval
+ Last_Modified_date>Planned_Delivery_End_Date
+ Op_Has_A_Valid_Quote_Expiry_Date

Hiper parámetros: 
 - colsample_bylevel: 0.7,
 - colsample_bytree: 0.7,
 - gamma: 0.0,
 - learning_rate: 0.11,
 - max_depth: 6,
 - min_child_weight: 1,
 - n_estimators: 95,
 - scale_pos_weight: 2,
 - subsample: 0.7000000000000002

**Resultado Kaggle = 0,54376**

# Carga de los data sets de test y de entrenamiento para LightGBM

In [57]:
#X = DATA , Y = TARGET 
X1, y1 = train_set_features[['Total_Taxable_Amount_USD',
                            'Bureaucratic_Code_0_Approved', 
                            'Products_Per_Op',
                            'TRF_Per_Op',
                            'Daily_Dif_Account_Created_Opportunity_Created_Date',
                            'Daily_Dif_Delivery_Start_Delivery_End_Date',
                            'Opp_Owner_Did_Last_Modifiy',
                            'Pricing, Delivery_Terms_Approved',
                            'Bureaucratic_Code_0_Approval',
                            'Last_Modified_date>Planned_Delivery_End_Date',
                            'Op_Has_A_Valid_Quote_Expiry_Date']], train_set_features['Stage']

Dividimos el set de entrenamiento en train y test, utilizando una proporción del 30% del total de los datos para el segundo.

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, random_state=45)

# Random Search con Light GBM

In [59]:
import lightgbm as lgb

In [60]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

Usamos como técnica de boosting: gbdt (gradient boosted decision trees).

In [61]:
lgb_clas = lgb.LGBMClassifier(boosting_type = 'gbdt', 
                                   eval_metric = 'logloss',
                                   verbose_eval=20)

In [62]:
posibles_hiperparametros_lgbm = {'learning_rate':np.arange(0.01,0.3,0.05),
                              'n_estimators':np.arange(16,116,5),
                              'scale_pos_weight':np.arange(2,6,1),
                              'max_depth':np.arange(2,7,1),
                            'min_child_weight':np.arange(1,10,1),
                              'gamma':np.arange(0,0.5,0.1),
                              'subsample':np.arange(0.4,0.8,0.1),
                            'colsample_bytree':np.arange(0.3,0.8,0.05),
                              'colsample_bylevel':np.arange(0.3,0.8,0.05)}

random_hiperparametros_lgbm = RandomizedSearchCV(lgb_clas, 
                                            posibles_hiperparametros_lgbm, 
                                            verbose = 3,
                                            n_jobs = -1,
                                            n_iter = 10000)

mejores_hp_lgbm = random_hiperparametros_lgbm.fit(X_train, y_train)

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-1)]: Done 508 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1148 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 1564 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2044 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 2588 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed:  8.2min
[Parallel(n_jobs=-1)]: Done 3868 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 4604 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done 5404 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 6268 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done 7196 tasks      | e

# Bagging Classifier con LightGBM

In [63]:
clf_lgbm = BaggingClassifier(base_estimator = lgb.LGBMClassifier(**mejores_hp_lgbm.best_params_),
                        n_estimators = 30, 
                        random_state = 0,
                        n_jobs = -1)

clf_lgbm.fit(X_train, y_train)

BaggingClassifier(base_estimator=LGBMClassifier(boosting_type='gbdt',
                                                class_weight=None,
                                                colsample_bylevel=0.44999999999999996,
                                                colsample_bytree=0.49999999999999994,
                                                gamma=0.1,
                                                importance_type='split',
                                                learning_rate=0.16000000000000003,
                                                max_depth=6,
                                                min_child_samples=20,
                                                min_child_weight=5,
                                                min_split_gain=0.0,
                                                n_estimators=106, n_jobs=-1,
                                                num_leaves=31, objective=None,
                                                rand

In [64]:
predicciones_train_lgbm = clf_lgbm.predict_proba(X_test)

In [65]:
from sklearn.metrics import log_loss
log_loss(y_test, predicciones_train_lgbm)

0.4397299731456451

In [66]:
predicciones_finales_lgbm = clf.predict_proba(test_set_features[['Total_Taxable_Amount_USD',
                            'Bureaucratic_Code_0_Approved', 
                            'Products_Per_Op',
                            'TRF_Per_Op',
                            'Daily_Dif_Account_Created_Opportunity_Created_Date',
                            'Daily_Dif_Delivery_Start_Delivery_End_Date',
                            'Opp_Owner_Did_Last_Modifiy',
                            'Pricing, Delivery_Terms_Approved',
                            'Bureaucratic_Code_0_Approval',
                            'Last_Modified_date>Planned_Delivery_End_Date',
                            'Op_Has_A_Valid_Quote_Expiry_Date']])

In [67]:
predicciones_exito_lgbm = []
for prob in predicciones_finales_lgbm:
  predicciones_exito_lgbm.append(prob[1])

In [68]:
csv_kaggle = pd.DataFrame()
csv_kaggle['Opportunity_ID'] = test_set_features["Opportunity_ID"]
csv_kaggle['target'] = predicciones_exito_lgbm

In [69]:
csv = csv_kaggle.to_csv('predicciones_bagging_lgbm.csv', index=False)
files.download('predicciones_bagging_lgbm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Resultados**

Algoritmo: LightGBM + Bagging

Features analizados:

+ Total_Taxable_Amount_USD 
+ Bureaucratic_Code_0_Approved
+ Products_Per_Op 
+ TRF_Per_Op 
+ Daily_Dif_Account_Created_Opportunity_Created_Date 
+ Daily_Dif_Delivery_Start_Delivery_End_Date 
+ Opp_Owner_Did_Last_Modifiy 
+ Bureaucratic_Code_0_Approval 
+ Pricing, Delivery_Terms_Approved 
+ Op_Has_A_Valid_Quote_Expiry_Date 
+ Last_Modified_date>Planned_Delivery_End_Date

Hiperparámetros:

Metodo: Random Search

- boosting_type='gbdt',
- class_weight=None,                                                - colsample_bylevel=0,49999999999
- colsample_bytree=0.499999999
- importance_type='split',
- learning_rate=016000000000000003,    
- max_depth=6,
- min_child_samples=20,
- min_child_weight=5,
- min_split_gain=0.0,                                                - n_estimators=106, n_jobs=-1,
- num_leaves=31, objective=None,
- random_state=None,                                                - reg_alpha=0.0,
- reg_lambda=0.0,               
- scale_pos_weight=3,
- silent=True,
- subsample=0.5,
- subsample_for_bin=200000,
- subsample_freq=0)


Resultado kaggle = 0.5443